# Math 579 Advanced Mathematical Analysis – Project
---
**Date:** 2025-04-21
**Name:** Samisoni Palu  
**Instructor:** Dr. Sun


In [1]:
import chess
import json

## Utils
Board data
---
`generate_uci_move_list`
- list all possible moves as a 2-tuple of from_square, to_square
- tweaked for promotion move counts
- will Include Redundant or Invalid Moves
    - **`a1a1`, `d4d4`**, etc. → no actual move is made
    - **Illegal under any real game condition**
    - Also includes nonsense like `h2h8` (rook-style moves for pawns)

But remember:  
> You’re not saying “these are all valid moves”  
> **this is the full move *vocabulary***—the possible *labels* in a classification task.
> That is, our **vocabulary** list is all the two-tuple pairings of squares, e.g. (`h2g3`)

**Benefits of the chosen Vocabulary**

1. Simplicity in Output Shape
    - 1-to-1 mapping: index ↔ UCI
    - You don’t need dynamic output heads or custom decoders
    - You can store logits as `torch.tensor([4672])` and just mask out illegal ones at runtime

2. Consistency
    - Your label space is fixed across:
      - Training
      - Inference
      - Evaluation

3. Non-moves Never Get Trained On
    - No master ever plays `a1a1`
    - So those output indices **never get gradient updates**
    - They just sit in the model—harmless dead neurons

**Why You Might Remove Redundant Moves**

1. Smaller Output Space
    - Saves compute on final linear layer and softmax
    - Slightly faster training (maybe)

2. Model Capacity Allocation
    - You force the network to **only ever consider valid move templates**
    - Could lead to sharper learning curve

But you pay with **more complexity**:
- Dynamic move indexing
- Pre-mask needs to align with training mask
- Harder debugging

---

## 🧠 Final Recommendation (For You Now)

> **Keep all UCI-formatted from-to combinations**, including non-moves like `a1a1`.

Why?

- You're just starting Phase 2
- You want clarity, consistency, and simplicity
- These junk moves won't hurt anything—they’ll just never activate

Later, if you *prove* they introduce drag or harm performance, you can refactor. But not now.  
Now is about **structure, execution, and momentum**.

Run with it.


In [2]:
def generate_uci_move_list():
    all_moves = set()
    board = chess.Board()
    for from_sq in chess.SQUARES:
        for to_sq in chess.SQUARES:
            move = chess.Move(from_sq, to_sq)
            all_moves.add(move.uci())
            # Add promotions
            for promo in [chess.QUEEN, chess.ROOK, chess.BISHOP, chess.KNIGHT]:
                promo_move = chess.Move(from_sq, to_sq, promotion=promo)
                all_moves.add(promo_move.uci())
    return sorted(all_moves)

def save_move_index_map(path="data/move_index_map.json"):
    moves = generate_uci_move_list()
    uci_to_index = {uci: i for i, uci in enumerate(moves)}
    with open(path, "w") as f:
        json.dump(uci_to_index, f)

In [7]:
print(type(chess.SQUARES[0]))
chess.Move(chess.SQUARES[0], chess.SQUARES[0])

<class 'int'>


Move.from_uci('0000')